<div class="report-header"><div class="aictx-logo"></div>
<span class="report-type">Demonstration</span><br />
<span class="report-author">Author: Felix Bauer</span><br />
<span class="report-date">25th January, 2020</span>
</div><h1>Live Demo:</h1><h1>ECG anomaly detection</h1>

This notebook demonstrates how recurrent SNNs can be used for anomaly detection in an electrocardiogram (ECG) signal. The main part of the network will run on a DYNAP-SE neuromorphic processor.

In [4]:
### --- Imports
import numpy as np
from matplotlib import pyplot as plt

### --- Constants
DT_ECG = 0.002778
NUM_ECG_LEADS = 2
NUM_ANOM_CLASSES = 5
MAX_FANIN = 64  # Max. number of presynaptic connections per neuron (hardware limit)

# Task

Our goal is to detect five different classes of anomalies in a two-lead ECG signal from the MIT-BIH Arrithmia Database [ref]. An SNN that fulfills this task can, for instance, be used in a wearable ECG monitoring device to trigger an alarm in presence of pathological patterns. Below we see examples for a normal ECG signal and for each anomaly type.

In [5]:
%matplotlib widget

from scripts.plot_example_beats import plot_examples, labels
plot_examples()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Hardware

We will first run a software simulation of our SNN and then run the network directly on a DYNAP-SE neuromorphic processor. The device we are demonstrating here is a prototype that imposes a few restrictions on the network, which are described below and which will also be considered in the software simulations.

## Core-wise parameters
Neuron and synapse parameters, such as time constants, firing thresholds and weights are set per core. The present processor consists of 16 cores of 256 neurons each. 

## Discrete weights
Synaptic weights are the same for each postsynaptic neuron on a core, resulting in ternary weights: positive (excitatory), negative (inhibitory), and zero (not connected). However, between each pair of neurons, multiple connections are possible, which effectively allows for integer weights.

## Connectivity
The number of presynaptic connections to each neuron is generally limited to 64.



# Input data

To load the ECG data we will use a data loader class that is specifically written for this purpose. You find its source code in the folder of this tutorial. 
The ECG data itself can be found at http://physionet.org/physiobank/database/mitdb/. We extracted the signal and its annotations to a .npy-file and a .csv file that can be found in ...

In [6]:
from scripts.dataloader import ECGDataLoader

# - Object to load ECG data
data_loader = ECGDataLoader()

ECG signal and annotaitions have been loaded from /home/felix/gitlab/Projects/AnomalyDetection/ECG/ecg_recordings


# Signal-to-spike encoding

The analog ECG signal is converted to trains of events through a sigma-delta encoding scheme. For every ECG lead there are two output channels, emitting spikes when the input signal increases ("up"-channel) or decreases ("down"-channel) by a specified amount.

The four resulting spike trains serve as input to the acutal network.

In [7]:
from rockpool.layers import FFUpDown

# - Spike encoding
spike_enc = FFUpDown(
    weights=NUM_ECG_LEADS,
    dt=DT_ECG,    
    thr_up=0.1,
    thr_down=0.1,
    multiplex_spikes=True,
    name="spike_encoder"
)

# Network architecture


To detect the anomalies we will use a reservoir network consisting of three layers:

## Input expansion layer

This layer consists of 128 neurons, each of which with up to 64 presynaptic excitatory connections (or a positive integer weight up to 64) to one of the four input channels. This connection scheme ensures that the neurons respond differently to the input, therefore increasing the dimensionality of the signal. 

This is enhanced by the fact that the hardware neurons on the DYNAP-SE slightly vary in their individual characteristics, which will result in richer neuron dynamics.

In [8]:
%matplotlib widget

# - Input expansion layer
num_ch_in = 2 * NUM_ECG_LEADS
size_expand = 128
baseweight_expand = 5e-4

# weights_expand = np.zeros((num_ch_in, size_expand))
# num_input_conns = np.random.randint(1, MAX_FANIN + 1, size=size_expand)
# num_neur_per_ext = int(np.floor(size_expand / num_ch_in))
# for idx_ch_in in range(num_ch_in):
#     idcs_inp = slice(idx_ch_in * num_neur_per_ext, (idx_ch_in + 1) * num_neur_per_ext)
#     weights_expand[idx_ch_in, idcs_inp] = num_input_conns[idcs_inp]
#
# np.save("weights/weights_expand.npy", weights_expand)
    
weights_expand = np.load("network/weights_expand.npy")

plt.imshow(weights_expand, aspect="auto")
plt.title("Weights of dimensionality expansion layer", fontsize=18)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Weights of dimensionality expansion layer')

## Reservoir layer

The reservoir layer consists of 512 excitatory and neuron 128 inhibitory neurons which are connected recurrently in a stochastic manner. Due to the recurrent connections, the network state at a given time does not only depend on the current input but also on previous network states, therefore implicitly encoding information about past inputs. This makes it possible to processs temporal relations in the input signal.

As a result, the state of each neuron in the reservoir is a function of the history of the input signal. If these functions are sufficiently independent from each other, they can be combined to approximate arbitrary functions. This linear combniation is done by the readout layer. In other words, the reservoir projects the signal into a high-dimensional state space, which ideally allows a linear separation of the different classes.

The partition into excitatory and inhibitory neurons is not strictly required but makes it easier to control neuron dynamics on the neuromorhpic processor.

In the following we will use a function provided by rockpool to generate the reservoir weights. For the excitatory recurrent connections the neurons are assumed to lie on a 2D grid and two neurons are more likely to be connected the closer they are to each other. Apart from that there are long-range connections which are independent of the neurons' positions on the grid.

Finally, we will assume the input expansion layer and the reservoir to be one layer. This is a technicality that makes possible the readout to access the neuron activities in the former layer as well.

In [9]:
from rockpool.weights import partitioned_2d_reservoir

## -- Reservoir layer
size_rec = 512
size_inh = 128
size_reservoir = size_rec + size_inh

# - Connections to excitatory layer
num_exp_rec = 16
num_inh = 16
# - Connections to inhibitory layer
num_rec_inh = 64
# - Recurrent connections
num_rec_short = 24  # Connections to neighbors
num_rec_long = 8  # Long-range connections

baseweight_exp_rec = 8e-5
baseweight_rec = 8e-5  # 1.75e-4
baseweight_rec_inh = 8e-5
baseweight_inh = 1e-4

# Fill expansion weights with 0s to fit full reservoir size
weights_res_in = np.hstack((weights_expand, np.zeros((num_ch_in, size_reservoir))))

# weights_rec = partitioned_2d_reservoir(
#     size_in=size_expand,
#     size_rec=size_rec,
#     size_inhib=size_inh,
#     max_fanin=MAX_FANIN,
#     num_inp_to_rec=num_exp_rec,
#     num_rec_to_inhib=num_rec_inh,
#     num_inhib_to_rec=num_inh,
#     num_rec_short=num_rec_short,
#     num_rec_long=num_rec_long,
#     width_neighbour=(2.0, 2.0),
# )
# np.save("weights/weights_rec.npy", weights_rec)

weights_rec = np.load("network/weights_rec.npy")

# Scale weights for software simulation
weights_res_in_scaled = weights_res_in.copy() * baseweight_expand

start_rec = size_expand
start_inh = size_expand + size_rec

weights_rec_scaled = weights_rec.copy()

weights_rec_scaled[:start_rec, start_rec: start_inh] *= baseweight_exp_rec
weights_rec_scaled[start_rec: start_inh, start_rec: start_inh] *= baseweight_rec
weights_rec_scaled[start_rec: start_inh, start_inh:] *= baseweight_rec_inh
weights_rec_scaled[start_inh:, start_rec: start_inh] *= baseweight_inh

In [10]:
%matplotlib widget
plt.imshow(weights_res_in_scaled, aspect="auto")
plt.title("Connections from expansion layer to reservoir")
plt.xlabel("Postsynaptic neurons")
plt.ylabel("Presynaptic neurons")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Presynaptic neurons')

In [11]:
%matplotlib widget
plt.imshow(weights_rec_scaled, aspect="auto")
plt.title("Recurrent reservoir connections")
plt.xlabel("Postsynaptic neurons")
plt.ylabel("Presynaptic neurons")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Presynaptic neurons')

In [14]:
from rockpool.layers import RecIAFSpkInNest

# - Load reservoir parameters from file (generated with gen_params.py)
kwargs_reservoir = dict(np.load("network/kwargs_reservoir.npz"))

# - Instantiate reservoir layer object
reservoir = RecIAFSpkInNest(
    weights_in=weights_res_in,
    weights_rec=weights_rec,
    name="reservoir",
    **kwargs_reservoir
)

## Readout layer

The readout layer low-pass filters the reservoir spike trains to obtain an analog signal. It is then trained by ridge regression to perform a linear separation between the ECG anomaly types. There is one readout unit for each anomaly and the corresponding target is 1 whenever the anomaly is present and 0 otherwise.

Although the linear regression algorithm is traditionally not for classification tasks, we use it here because it is very fast and efficient.

In [54]:
from rockpool.layers import FFExpSyn

readout = FFExpSyn(
    weights=np.zeros((size_full, NUM_ANOM_CLASSES)),
    bias=0,
    dt=DT_ECG,
    tau_syn=0.175,
    name="readout"
)

## Network

In [63]:
from rockpool import Network

# - Network that holds the layers
sw_net = Network(spike_enc, reservoir, readout, dt=DT_ECG)

# Software Simulation

## Training
In the following we will train the (software) readout layer. For this we generate batches of ECG data and evolve the network with it as input. After each batch the readout weights are updated.

We have trained the readout beforehand and will simply load the weights.

In [64]:
# import time

# # - Generator that yields batches of ECG data
# batchsize_training = 1000
# num_beats = 10000
# regularize = 0.1
# batch_gen = data_loader.get_batch_generator(num_beats=num_beats, batchsize=batchsize_training)

# t_start = time.time()
# for batch in batch_gen:
#     output = sw_net.evolve(batch.input)
#     readout.train_rr(
#         batch.target,
#         output["reservoir"],
#         is_first=batch.is_first,
#         is_last=batch.is_last,
#         regularize=regularize,
#     )
    
# sw_net.reset_all()
# print(f"Trained network in {time.time() - t_start:.2f} seconds.")

# np.save("network/readout_weights", readout.weights)
# np.save("network/readout_bias", readout.bias)

In [69]:
# # - Test on training data
# target = batch.target.start_at_zero()
# res_data = output["reservoir"]

# test_on_training = readout.evolve(res_data.start_at_zero())
# readout.reset_all()

In [70]:
# %matplotlib widget
# from rockpool import TSContinuous
# test_on_training.clip(channels=4).plot()
# target.clip(channels=4).plot()
# any_target = TSContinuous(target.times, np.any(target.samples, axis=1))
# any_target.plot(color="gray", alpha=0.5)

In [15]:
readout.weights = np.load("network/readout_weights.npy")
readout.bias = np.load("network/readout_bias.npy")

NameError: name 'readout' is not defined

## Inference

We can now test our network to see how it performs with data it has not been trained on.
The plots below show the output of each readout unit (blue). The targets are plotted in orange. 

Because the readout units are only trained to distinguish between normal and one specific anomaly, there many cross-detections. We therefore also plot a gray curve that indicates the presence of _any_ anomaly. For many applications it is sufficient to know that there is an anomaly. If one needs to classify which type it is, one could, for instnace, train an all-vs-all classifier.

In [16]:
# - Generator that yields batches of ECG data
num_beats = 100
ecg_data = data_loader.get_single_batch(num_beats=num_beats)

net_data = sw_net.evolve(ecg_data.input)
output = net_data["readout"]
sw_net.reset_all()

NameError: name 'sw_net' is not defined

In [72]:
%matplotlib widget

target = ecg_data.target
any_target = TSContinuous(target.times, np.any(target.samples, axis=1))

fig, axes = plt.subplots(3, 2, figsize=(8, 10))
axes[-1, -1].set_visible(False)
plt.subplots_adjust(
    top=0.98, bottom=0.05, left=0.15, right=0.95, hspace=0.5, wspace=0.2
)

for i_anom, (ax, lbl) in enumerate(zip(axes.flatten()[:-1], labels)):
    output.clip(channels=i_anom).plot(target=ax)
    any_target.plot(target=ax, color="gray", alpha=0.5)
    target.clip(channels=i_anom).plot(target=ax)
    ax.set_title(lbl)
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Hardware Implementation

Now it is time to replace the software reservoir with the neuromorphic processor. As weights, we can simply use the (unscaled) integer weights from which we also generated the weights of the software reservoir.

Neuron and synapse parameters will be loaded from a file directly onto the chip. They are chosen so that the resulting dynamics are close to that of the simulation.

Note that the hardware parameters are often refered to as "biases", because they correspond to biases in circuits on the chip. They are not to be confused with the bias of a neuron in a neural network.

After the parameters have been set, all neurons should be quiet. Sometimes there are "hot" neurons, which keep firing anyway. We will identify those and simply disable them.

## Imports and settings

In [76]:
# - Imports and parameters
from rockpool.devices import rectangular_neuron_arrangement, DynapseControlExtd
from rockpool.layers import RecDynapSE

# Path for loading circuit biases (which define neuron and synapse characteristics)
bias_path = "network/biases.py"

# How long to scan for 'hot' neurons that fire spontaneously
silence_hot_neurons_dur = 5

## Neuron arrangement

We can choose explicitely to which individual neurons on the chip the neurons from the network are mapped to. It makes sense to put different partitions of the reservoir (input expansion, excitatory, inhibitory) on different cores, so that the neuron dynamics can be adjusted individually.

Furthermore, the neurons will be assigned so they form rectangles. This makes it easier to visually identify individual neurons in cortexcontrol, the software interface to the chip.

We also need to select virtual neurons, that act as a source for the external spikes from the signal-to-spike layer. The important thing is that they should have different IDs than the hardware neurons.

In [78]:
# - Reservoir neuron arangement
rectangular_arrangement = [
    # Input layer
    {"first_neuron": 4, "num_neurons": size_in, "width": 8},
    # Reservoir layer I
    {"first_neuron": 256, "num_neurons": 256, "width": 16},
    # Reservoir layer II
    {"first_neuron": 768, "num_neurons": size_rec - 256, "width": 16},
    # Inhibitory layer
    {"first_neuron": 516, "num_neurons": size_inhib, "width": 8},
]

neuron_ids = []
for rectangle_params in rectangular_arrangement:
    neuron_ids += list(rectangular_neuron_arrangement(**rectangle_params))
    
# - 'Virtual' neurons (input neurons)
virtual_neuron_ids = [1, 2, 3, 12]

In [79]:
# - Set up DynapseControl

controller = DynapseControlExtd(fpga_isibase=reservoir.dt)

# Circuit biases
controller.load_biases(bias_path)

# Silence 'hot' neurons that fire continuously
hot_neurons = controller.silence_hot_neurons(neuron_ids, silence_hot_neurons_dur)

dynapse_control: RPyC connection established through port 1300.
dynapse_control: RPyC namespace complete.
dynapse_control: RPyC connection has been setup successfully.
DynapseControl: Initializing DynapSE
DynapseControl: Spike generator module ready.
DynapseControl: Poisson generator module ready.
DynapseControl: Time constants of cores [] have been reset.
DynapseControl: Neurons initialized.
	 0 hardware neurons and 1023 virtual neurons available.
DynapseControl: Neuron connector initialized
DynapseControl: Connectivity array initialized
DynapseControl: FPGA spike generator prepared.
DynapseControl ready.
DynapseControl: Biases have been loaded from biases.py.
DynapseControl: Collecting IDs of neurons that spike within the next 5 seconds
DynapseControl: Generated new buffered event filter.
DynapseControl: 28 neurons spiked: [293, 302, 303, 335, 355, 364, 420, 428, 453, 523, 536, 570, 585, 601, 618, 681, 709, 761, 825, 836, 851, 853, 910, 942, 953, 986, 990, 1009]
DynapseControl: Neuro

In [80]:
# - Set up hardware reservoir layer
hw_layer = RecDynapSE(
    weights_in=weights_res_in,
    weights_rec=weights_rec,
    neuron_ids=neuron_ids,
    virtual_neuron_ids=virtual_neuron_ids,
    dt=reservoir.dt,
    controller=controller,
    clearcores_list=[0,1,2,3],
    name="hardware",
)

RecDynapSE `hardware`: Superclass initialized
DynapseControl: Connections to cores [0 1 2 3] have been cleared.
DynapseControl: For some of the requested neurons, chips need to be prepared.
dynapse_control: Chips 0 have been cleared.
DynapseControl: 1023 hardware neurons available.
Layer `hardware`: Layer neurons allocated
Layer `hardware`: Virtual neurons allocated
DynapseControl: Excitatory connections of type `FAST_EXC` between virtual and hardware neurons have been set.
DynapseControl: Inhibitory connections of type `FAST_INH` between virtual and hardware neurons have been set.
Layer `hardware`: Connections to virtual neurons have been set.
DynapseControl: Excitatory connections of type `FAST_EXC` between hardware neurons have been set.
DynapseControl: Inhibitory connections of type `FAST_INH` between hardware neurons have been set.
Layer `hardware`: Connections from input neurons to reservoir have been set.
DynapseControl: Excitatory connections of type `SLOW_EXC` between hardware

In [81]:
# - Separate readout layer (for different readout weights)
readout_hw = FFExpSyn(
    weights=np.zeros((size_full, NUM_ANOM_CLASSES)),
    bias=0,
    dt=DT_ECG,
    tau_syn=0.175,
    name="readout"
)

In [83]:
sw_net.reset_all()
hw_net = Network(spike_enc, hw_layer, readout_hw, dt=DT_ECG)

## Training

In [2]:
# # - Generator that yields batches of ECG data
# batch_gen = data_loader.get_batch_generator(num_beats=100, batchsize=batchsize_training)

# for batch in batch_gen:
#     output = hw_net.evolve(batch.input)
#     readout_hw.train_rr(
#         batch.target,
#         output["hardware"],
#         is_first=batch.is_first,
#         is_last=batch.is_last,
#         regularize=regularize,
#     )
# hw_net.reset_all()

# np.save("weights/readout_weights_hw.py", readout_hw.weights)
# np.save("weights/readout_bias_hw.py", readout_hw.bias)

In [ ]:
# - Load pre-trained weights
readout_hw.weights = np.load("network/readout_weights_hw.py")
readout_hw.bias = np.load("network/readout_bias_hw.py")

## Inference

In [ ]:
# - Generator that yields batches of ECG data
num_beats = 100
ecg_data = data_loader.get_single_batch(num_beats=num_beats)

net_data = hw_net.evolve(ecg_data.input)
output = net_data["readout"]

In [56]:
%matplotlib widget

target = ecg_data.target
any_target = TSContinuous(target.times, np.any(target.samples, axis=1))

fig, axes = plt.subplots(3, 2, figsize=(8, 10))

plt.subplots_adjust(
    top=0.98, bottom=0.05, left=0.15, right=0.95, hspace=0.5, wspace=0.2
)

for i_anom, (ax, lbl) in enumerate(zip(axes.flatten()[:-1], labels)):
    output.clip(channels=i_anom).plot(target=ax)
    any_target.plot(target=ax, color="gray", alpha=0.5)
    target.clip(channels=i_anom).plot(target=ax)
    ax.set_title(lbl)
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …